In [2]:
# Import required libraries
import csv
import requests
import pandas as pd
from datetime import datetime
import os
from dune_client.client import DuneClient

### Fetch the results from Dune

In [2]:
api_key = "WPWyWJJZEPTxvOwQmpcHbfbHoqXHaWxv"  # Replace with your Dune API key
query_id = 5301108  # Replace with your query ID

# Initialize the DuneClient
dune = DuneClient(api_key)

# Function to fetch Dune query results
def fetch_dune_query_results(query_id):
    try:
        query_result = dune.get_latest_result(query_id)
        return query_result.result.rows  # Access the rows in the response
    except Exception as e:
        print(f"Error fetching query results: {e}")
        return None

# Function to write results to CSV
def write_to_csv(data, file_name):
    try:
        # Open file in write mode
        with open(file_name, mode='w', newline='') as file:
            writer = csv.writer(file)
            
            # Write headers (keys of the first row)
            if len(data) > 0:
                headers = data[0].keys()  # Get keys from the first row
                writer.writerow(headers)
                
                # Write rows (values)
                for row in data:
                    writer.writerow(row.values())
        print(f"Data written to {file_name}")
    except Exception as e:
        print(f"Error writing to CSV: {e}")

# Main code to fetch and save results
if __name__ == "__main__":
    result = fetch_dune_query_results(query_id)
    if result:
        file_name = "../Data/Active_Delegates_Data.csv"
        write_to_csv(result, file_name)

Data written to ../Data/Active_Delegates_Data.csv


### Fetch the delegates from the Subgraph

In [3]:
def fetch_top_500_delegates():
    """
    Fetches the top 200 delegates sorted by latestBalance in descending order.
    Returns:
        pd.DataFrame: A DataFrame containing delegate address and voting power.
        str: Filename where data is saved.
    """
    url = "https://api.studio.thegraph.com/query/68573/op/v0.0.9"

    query = """
    query {
      delegates(first: 500, orderBy: latestBalance, orderDirection: desc) {
        id
        latestBalance
      }
    }
    """

    try:
        response = requests.post(url, json={"query": query})
        response.raise_for_status()
        result = response.json()

        if "errors" in result:
            raise ValueError(f"GraphQL Error: {result['errors']}")

        data = result.get("data", {}).get("delegates", [])
        if not data:
            print("No data returned.")
            return pd.DataFrame(), None

        # Convert to DataFrame and process
        df = pd.DataFrame(data)
        df.rename(columns={"id": "delegate", "latestBalance": "voting_power"}, inplace=True)
        df["voting_power"] = df["voting_power"].astype(float) / 1e18

        # Save to CSV
        current_date = datetime.now().strftime("%Y-%m-%d")
        file = "../Data/Active_Delegates_Data.csv"
        df.to_csv(file, index=False)
        print(f"Data saved to {file}")

        return df

    except Exception as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame(), None

df = fetch_top_500_delegates()


Data saved to ../Data/Active_Delegates_Data.csv


In [4]:
delegates_df = pd.read_csv("../Data/Active_Delegates_Data.csv")

In [5]:
delegates_df

,delegate,voting_power
0,0x3eee61b92c36e97be6319bf9096a1ac3c04a1466,1.105058e+07
1,0x2ac5393d1f4be4ef89b45ee2f93d7f20a5cf6d5a,7.026108e+06
2,0x06ad892ce23c136bbda3a821570343a2af3e2914,5.557493e+06
3,0xf11b6a8c3cb8bb7dbc1518a613b10ceb0bbfc06b,5.221222e+06
4,0x1b686ee8e31c5959d9f5bbd8122a58682788eead,5.041244e+06
...,...,...
495,0xf42ad66f4b66628584f2e97421a008d62a8f116c,3.018360e+03
496,0x8b78d14e242ccba658af9455df6c021f2e4e994f,3.002746e+03
497,0x5b93ff82faaf241c15997ea3975419dddd8362c5,2.963086e+03
498,0xd1c416489922301dbacad84d177c3d750720c81c,2.947680e+03
